In [400]:
#Decision Tree Implementation with "Pseudo" Feature Embedding
#Huy Dinh Tran
#Reference: https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/ 

#Importing dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import tree
from IPython.display import Image, display
import matplotlib.pyplot as plt
import pydotplus
import matplotlib.image as pltimg
from dtreeviz.trees import dtreeviz
from sklearn.metrics import accuracy_score


#### Decision Tree Classification Model from scratch

In [401]:
#Node Class
#Each node holds feature index, threshold, left and right output, and info gain value
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [402]:
#Tree Class
class DecisionTreeClassifier():

    #initializing the tree
    def __init__(self, min_samples_split, max_depth): 
        self.root = None
        
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
    
    #Recursion function using other functions to build an optimal tree
    def build_tree(self, dataset, curr_depth=0):        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    #finding the best split after trying different configurations
    def get_best_split(self, dataset, num_samples, num_features):        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    #spliting the data left and right
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    #output info gain value
    def information_gain(self, parent, l_child, r_child, mode="entropy"):        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini(parent) - (weight_l*self.gini(l_child) + weight_r*self.gini(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    #output entropy value
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    #output gini value
    def gini(self, y):        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    #output leaf node
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)
    
    #print dicision tree
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    #train decision tree model
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    #test/predict new dataset
    def predict(self, X):        
        preditions = [self.one_prediction(x, self.root) for x in X]
        return preditions

    #predict only one data point
    def one_prediction(self, x, tree):        
        if tree.value != None: 
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.one_prediction(x, tree.left)
        else:
            return self.one_prediction(x, tree.right)

#### Training and Testing

In [403]:
#Reading and spliting our dataset to training and testing data
trainingData = pd.read_csv("../../Data/Dataset.csv")
testingData = pd.read_csv("../../Data/TestingData.csv").values

targets = trainingData["home_team_result"]
features = trainingData.drop(["home_team_result","winner_encoded"],axis=1).values

featuresTrain, targetsTrain = features[0:48], targets[0:48]
featuresTest, targetsTest = testingData, targets[48:]
targetsTrain=targetsTrain.values.reshape(-1,1)

In [404]:
#Training
dtree = DecisionTreeClassifier(min_samples_split=20, max_depth=20)
dtree.fit(featuresTrain, targetsTrain)

#Testing
test = dtree.predict(featuresTest) 
print('Accuracy: ',accuracy_score(targetsTest, test)*100,'%')

Accuracy:  50.0 %


In [405]:
#print decision tree
dtree.print_tree()

X_2 <= 1.0 ? 0.2556991882774616
 left:X_3 <= 0.0 ? 0.2501400784439287
  left:1.0
  right:X_8 <= 3.0 ? 0.0355029585798817
    left:0.0
    right:0.0
 right:1.0
